In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [11]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [4]:
import logging
from autogen_core import EVENT_LOGGER_NAME

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [12]:
from autogen_core.models import UserMessage

result = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)


{"tools": [], "type": "LLMCall", "messages": [{"role": "user", "name": "user", "content": "What is the capital of France?"}], "response": {"id": "chatcmpl-CpvOVZdJWfcLMBZVgItpXzxz5CJ1g", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "The capital of France is Paris.", "refusal": null, "role": "assistant", "annotations": [], "audio": null, "function_call": null, "tool_calls": null}}], "created": 1766491387, "model": "gpt-4o-mini-2024-07-18", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_c4585b5b9c", "usage": {"completion_tokens": 7, "prompt_tokens": 15, "total_tokens": 22, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}, "prompt_tokens": 15, "completion_tokens": 7, "agent_id": null}
INFO:autogen_core.events:{"tools": [], "type": "LLMCall", "messages": [

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None


In [16]:
from autogen_agentchat.messages import TextMessage

message = TextMessage(content="I'd like to go to Messina", source="user")
message

TextMessage(id='2f3ce2af-ba97-4e39-aded-8a48392efb10', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 12, 23, 12, 6, 12, 191993, tzinfo=datetime.timezone.utc), content="I'd like to go to Messina", type='TextMessage')

In [17]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. you give short, humorous answer.",
    model_client_stream=True
)

In [18]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

{"type": "LLMStreamStart", "messages": [{"content": "You are a helpful assistant for an airline. you give short, humorous answer.", "role": "system"}, {"role": "user", "name": "user", "content": "I'd like to go to Messina"}], "agent_id": null}
INFO:autogen_core.events:{"type": "LLMStreamStart", "messages": [{"content": "You are a helpful assistant for an airline. you give short, humorous answer.", "role": "system"}, {"role": "user", "name": "user", "content": "I'd like to go to Messina"}], "agent_id": null}
{"type": "LLMStreamEnd", "response": {"finish_reason": "stop", "content": "Well, grab your snacks and put on a sun hat\u2014Messina's calling! Just don\u2019t try to \"mess in-a\" the flight plans!", "usage": {"prompt_tokens": 0, "completion_tokens": 0}, "cached": false, "logprobs": null, "thought": null}, "prompt_tokens": 0, "completion_tokens": 0, "agent_id": null}
INFO:autogen_core.events:{"type": "LLMStreamEnd", "response": {"finish_reason": "stop", "content": "Well, grab your s

'Well, grab your snacks and put on a sun hat—Messina\'s calling! Just don’t try to "mess in-a" the flight plans!'

In [19]:
import os
import sqlite3

if os.path.exists("tickets.db"):
    os.remove("tickets.db")

conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [ ]:
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

In [21]:
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [30]:
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [31]:
get_city_price("Rome")

499.0

In [32]:
smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    tools=[get_city_price],
    reflect_on_tool_use=True,
    system_message="You are a helpful assistant for an airline. you give short, humorous answer.",
    model_client_stream=True,
)

In [33]:
new_message = TextMessage(content="How much is the price ticket for a round trip to rome?", source="user")

In [36]:
response = await smart_agent.on_messages([new_message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)

response.chat_message.content

{"type": "LLMStreamStart", "messages": [{"content": "You are a helpful assistant for an airline. you give short, humorous answer.", "role": "system"}, {"role": "user", "name": "user", "content": "How much is the price ticket for a round trip to rome?"}, {"role": "assistant", "tool_calls": [{"id": "call_wlOvw2C3qYAOY5mGiDo5Ui5j", "function": {"arguments": "{\"city_name\":\"Rome\"}", "name": "get_city_price"}, "type": "function"}], "content": null}, {"content": "499.0", "role": "tool", "tool_call_id": "call_wlOvw2C3qYAOY5mGiDo5Ui5j"}, {"role": "assistant", "content": "A round trip to Rome costs about $499... or the equivalent of 500 slices of pizza (but we recommend the ticket)! \ud83c\udf55\u2708\ufe0f"}, {"role": "user", "name": "user", "content": "How much is the price ticket for a round trip to rome?"}], "agent_id": null}
INFO:autogen_core.events:{"type": "LLMStreamStart", "messages": [{"content": "You are a helpful assistant for an airline. you give short, humorous answer.", "role":

'A round trip to Rome will cost you about $499. That’s cheaper than a night out with the Pope! 🕊️✈️'

In [35]:
response

Response(chat_message=TextMessage(id='da6ec1bd-3326-426b-9978-6957e41c95bb', source='smart_airline_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 12, 23, 12, 31, 4, 977577, tzinfo=datetime.timezone.utc), content='A round trip to Rome costs about $499... or the equivalent of 500 slices of pizza (but we recommend the ticket)! 🍕✈️', type='TextMessage'), inner_messages=[ToolCallRequestEvent(id='d07a981d-465c-409a-b438-2488c72e4ff1', source='smart_airline_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 12, 23, 12, 31, 2, 132354, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_wlOvw2C3qYAOY5mGiDo5Ui5j', arguments='{"city_name":"Rome"}', name='get_city_price')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='42e0f073-fe10-4a06-9b2d-3925c6dd8756', source='smart_airline_agent', models_usage=None, metadata={}, created_at=dateti